# 1교시

알아서 기본 세팅은 잡아라.

```python
# /blog/models.py 파일
from django.db import models
from django.utils import timezone

class Post(models.Model):
    author = models.ForeignKey('auth.User', on_delete=models.CASCADE)
    title = models.CharField(max_length=200)
    text = models.TextField()  # 글자수에 제한 없는 텍스트
    created_date = models.DateTimeField(
        default=timezone.now)  # 날짜와 시간
    published_date = models.DateTimeField(
        blank=True, null=True) #  필드가 폼에서 빈 채로 저장되는 것을 허용

    def publish(self):
        self.published_date = timezone.now()
        self.save()

    def __str__(self):
        return self.title
```

`author = models.ForeignKey('auth.User', on_delete=models.CASCADE)`: 아마도 auth에 등록된 유저 정보를 사용하겠다는 의미인 것 같은데 질문하셈!!!

`def publish(self):...`: 아마 `publish_date`를 빈 값으로 내버려둔채 저장할 시 저장시점을 `publish_date`로 설정하기 위함인 것 같은데 질문하셈!!!

```python
# /blog/admin.py 파일
from django.contrib import admin
from blog.models import Post

admin.site.register(Post)
```

`admin.site.register(Post)`: 이 코드를 작성해야 localhost:8000/admin 접속 시 Post의 DB를 확인할 수 있다.

```python
# /blog/urls.py 파일
from django.urls import path
from . import views

urlpatterns = [
    path('', views.index),
]
```

```python
# /blog/views.py 파일
from django.shortcuts import render
from django.http import HttpResponse
from blog.models import Post

def index(req):
    return HttpResponse('이번에는 제발 되게 해주세요...')
```

* 동적 주소 생성

```python
# /blog/views.py 파일
def index2(req, name):
    return HttpResponse(name + '님 환영합니다.')
```

```python
# /blog/urls.py 파일
urlpatterns = [
    path('', views.index),
    path('<name>', views.index2),
]
```

.../blog/'아무거나(알파벳이긴 해야함)'로 접속 시, ''아무거나'님 환영합니다.'라는 메세지를 확인할 수 있다.  
이처럼 주소에 입력된 값을 params로 처리하여 보여주기 위해서는 '<>'이 필요하다.  
또한 views.py의 정의된 함수(index2)에 인자도 추가해 주어야 한다.

```python
# /blog/urls.py 파일
urlpatterns = [
    path('', views.index),
    path('<name>', views.index2),
    path('<int:pk>/detail', views.index3),
]
```

```python
# /blog/views.py 파일
from django.shortcuts import get_object_or_404

def index3(req, pk):
    # p = Post.objects.get(pk=pk)
    p = get_object_or_404(Post, pk=pk)
    return HttpResponse('제목: ' + p.title)
```

`path('<int:pk>/detail', views.index3),`: <int:pk>는 pk라는 parameter를 받는데 그 데이터 타입이 정수형임을 의미한다. 예를들어 .../blog/18/detail 주소를 의미한다.

`p = get_object_or_404(Post, pk=pk)`
`get_object_or_404()`: client에서 pk 특정 값으로 호출할 때, 만약 DB에 해당 값이 존재하지 않는다면 404 메세지를 return하는 함수이다.  
이전에는 주석처리된 코드처럼 `get()`을 이용하여 호출했다. 그러나 이러한 방식은 해당 값이 DB에 존재하지 않을 때 에러가 발생한다. default 값을 설정하여 에러 발생을 방지할 수 있지만, 이는 대표성을 지닌 data가 존재해야한다는 단점이 있다.  
예를들어 이번에 구현하려는 페이지는 총 3개의 글(pk=1~3)이 있는 Post DB에 접근해 client가 원하는 글을 보여주는 것이 목적이다. 하지만 client가 .../blog/4/detail으로 접속하면 404 페이지를 마주하게 된다.

* 게시글을 리스트로 보여주는 페이지

```python
# /blog/urls.py 파일
urlpatterns = [
    path('', views.index),
    path('<name>', views.index2),
    path('<int:pk>/detail', views.index3),
    path('Blist/', views.Blist),
]
```

`path('Blist/', views.Blist),`: 항상 '/'를 붙이는 습관을 들여야할 것 같다. 그냥 'Blist'라고만 작성하면 접근 몬함.

```python
# /blog/views.py 파일
def Blist(req):
    data = Post.objects.all()
    return render(req, 'blog/Blist.html', {'data':data})
```

`Post.objects.all()`: DB에 있는 'blog_post'(=Post) 테이블의 모든 데이터를 호출.

```html
<!-- /templates/blog/list.html 파일 -->
<h2> Posts </h2> <br>

{% for d in data %}
    {{d.title}}<br>
    {{d.text}}<br>
    <hr> 
{% endfor %}
```

`{% for d in data %}`: views.py(Blist)에서 data 인자를 넘겨 받아 하나씩(=d) for문을 수행.

`{{d.title}}<br>`: d에서 title이라는 key를 가진 value를 출력. 기본적으로 Python의 dictionary와 동일하다.

`<br>`: 줄 긋기. Markdown의 `---`과 같다.

* 사이트 전반에 걸친 공통 템플릿 만들기

사이트 전반에 걸쳐 항상 보여지는 부분들을(상단의 메뉴바, 하단의 기업 소재지/연락처와 같은 정보) 따로 html로 작성한 후 모든 페이지에 보여줄 수 있다.

이를 위해 항상 보여지는 부분을 base template에 작성하고, 변하는 부분을 sub-template에 따로 작성한다. sub-template에서 base template을 상속 받을 수 있기에 가능하다.

```html
<!-- /templates/blog/base.html 파일 -->
<h1>Blog</h1>
<hr size=5 color='blue'>

{% block content %}
{% endblock %}

<hr size=5 color='blue'>
[기업 정보]<br>
- 대표자명: Bandai Namco<br>
- 주소: Japan<br>
```

`{% block content %}...{% endblock %}`: 이 부분에 페이지 마다 달라지는 내용이 들어간다.

그 외의 내용들은 공통적으로 보여지는 내용이다.

```html
<!-- /templates/blog/list.html 파일 -->
{% extends 'blog/base.html' %}

{% block content %}

    <h2> Posts </h2> <br>

    {% for d in data %}
        {{d.title}}<br>
        {{d.text}}<br>
        <hr>
    {% endfor %}

{% endblock %}
```

`{% extends 'blog/base.html' %}`: base.html을 상속 받는다.

`{% block content %}...{% endblock %}`: 이 부분을 해당 코드로 치환한다.

* 개별 포스트 보여주기

Blist 페이지에서는 전체 포스트의 제목만 보여주고, 제목을 클릭하면 그 포스트의 내용을 보여주는 기능을 구현하려고 한다.

```html
<!-- /templates/blog/list.html 파일 -->
{% for d in data %}
        <a href='/blog/{{d.pk}}/detailPost'>{{d.title}}</a><br>
    {% endfor %}
```

제목만 보여주기 위해 d.text 코드를 제거하였다.

`<a href='/blog/{{d.pk}}/detailPost'>...</a>`: 제목 클릭시 해당 포스트로 이동하기 위해 앵커 태그(`<a>...</a>`)를 사용하였다. 앵커 태그 사용시 '/'로 시작한다면 절대 경로가 된다. 따라서 해당 포스트를 보여주는 주소 .../blog/18(='해당 포스트의 pk 값')/detailPost 로 연결 된다.

```html
<!-- /templates/blog/detailPost.html 파일 -->
{% extends 'blog/base.html' %}

{% block content %}

    <h2>Detail post</h2><br>
    제목: {{pk.title}}<br><br>
    내용: {{pk.text|linebreaks}}

{% endblock%}
```

`{{pk.text|linebreaks}}`: 줄바꿈인 엔터(\n)가 html에서는 의미가 없으므로, 이를 `<br>`로 치환해주기 위해 '|linebreaks'를 사용한다.

```python
# /blog/views.py 파일
def detailPost(req, pk):
    p = get_object_or_404(Post, pk=pk)
    return render(req, 'blog/detailPost.html', {'pk':p})
```

```python
# /blog/urls.py 파일
urlpatterns = [
    path('', views.index),
    # path('<name>', views.index2),
    path('<int:pk>/detail', views.index3),
    path('Blist', views.Blist),
    path('<int:pk>/detailPost', views.detailPost),
]
```

* class 객체로 코딩하여 페이지 보여주기

```python
# /blog/views.py 파일
from django.views.generic import View

class klass(View):
    def get(self, req):
        return HttpResponse('GET 방식으로 요청하셨습니다.')
    def post(self, req):
        return HttpResponse('POST 방식으로 요청하셨습니다.')
```

```python
# /blog/urls.py 파일
urlpatterns = [
    path('', views.index),
    # path('<name>', views.index2),
    path('<int:pk>/detail', views.index3),
    path('Blist/', views.Blist),
    path('<int:pk>/detailPost/', views.detailPost),
    path('klass/', views.klass.as_view()),
]
```

* class를 통한 로그인 기능 만들기: 200217과 차이점 비교해 보셈

```python
# /blog/views.py 파일
from django.shortcuts import redirect

class LoginView(View):
    def get(self, request):
        return render(request, 'blog/login.html')
    def post(self, request):
        return redirect('Blist')
```

`redirect(Blist)`: 특정 페이지(Blist)로 돌려보낸다.

일반적으로 get 방식의 요청은 render로 처리하고, post 방식의 요청은 redirect로 처리한다.

```python
# /blog/urls.py 파일
    path('login/', views.LoginView.as_view()),
```

```html
<!-- /templates/blog/login.html 파일 -->
<form action='' method='POST'>
    {% csrf_token %}
    ID : <input type=text name=username><br>
    P/W : <input type=password name=password>
    <input type=submit value='Login'>
</form>
```

`{% csrf_token %}`: POST 방식으로 보낼 때, 방화벽 같은거 열어달라는 소리. 구글에 csrf 검색하면 뭐 나옴.

물론 아직까지는 로그인을 시도해도 error 페이지만 나타난다.

* 로그인을 위한 인증 기능 구현

교과서에서 이와 관련한 내용은 10장(인증 기능)에 나타나고 있다.

교과서의 내용은 Django가 기본적으로 제공하는 app을 기반으로 코드를 작성하고 있지만 여기서는 기본 app의 함수(혹은 class)를 최소한으로 사용하려고 한다???

Django에서는 django.contrib.auth 앱이 인증 기능을 담당한다.

SQLite을 통해 auth_user 테이블의 데이터를 확인해보면, 비밀번호는 암호화 처리되어 있어, DB 관리자 조차 알 수 없다. 이를 해결하기 위해 Django가 제공하는 라이브러리(django.contrib.auth 내부에 있는)를 사용한다.

```python
# /blog/views.py 파일
class LoginView(View):
    def get(self, request):
        return render(request, 'blog/login.html')
    def post(self, request):
        un = request.POST.get('username')
        pw = request.POST.get('password')
        
        user = authenticate(username=un, password=pw)
        
        if user == None:
            return redirect('/blog/login')
        else:
            return redirect('/blog/Blist')
```

`un = request.POST.get('username')`: 이 코드를 말로 풀어 쓰면 다음과 같다. POST 방식의 요청이 들어왔을 때, 키가 'username'의 값을 un으로 저장한다.  
자세히 살펴보자. login.html에서 `<form action='' method=POST`였기 때문에 제출(submit) 시, POST 방식으로 요청한다는 것을 알 수 있다. 그리고 `ID: <input type=text name=username>`이었으므로, client가 ID에 입력한 값은 key가 username인 dictionary가 된다.

위의 내용을 설명할 때 자꾸 dictionary라는 것을 언급하는데, 이는 뭐 알지요? POST 방식의 요청이 JSON형식이고, 그게 파이썬에서는 결론적으로 dictionary랑 같은거구.... 이런 관련 이야기들.

`authenticate()`: 인증 기능.

`if user == None: ...redirect('/blog/Blist')`: authenticate 함수의 return 값이 None이면 비밀번호가 잘못된 것(물론 아이디가 없을 수도 있음)이므로 다시 로그인 페이지로 보낸다. 그 외의 경우에는 블로그의 전체 포스트를 볼 수 있는 페이지(Blist)로 보낸다.

* session 이용하기

로그인 후에는 일반적으로 로그인한 사용자에 맞춤 정보/기능을 제공된다. 이렇게 페이지를 옮겨 다녀도 로그인 상태가 유지되는 것은 session 기능 덕분이다(?) session에 대한 자세한 설명을 알아서 찾아보라.

이번 예제의 목표는 로그인 후 해당 사용자가 작성한 포스트만 보여주거나, 해당 사용자의 아이디로 포스트를 작성할 수 있게 만드는 것이다. 다만 이번 코드만으로는 완벽히 구현하지는 않는다.

```python
# /blog/views.py 파일
class LoginView(View):
    def get(self, request):
        return render(request, 'blog/login.html')
    def post(self, request):
        un = request.POST.get('username')
        pw = request.POST.get('password')
        
        user = authenticate(username=un, password=pw)
        
        if user == None:
            return redirect('/blog/login')
        else:
            request.session['username'] = un # 세션
            return redirect('/blog/Blist')

# /blog/views.py 파일. Blist 수정
def Blist(req):
    username = req.session.get('username', 'guest')
    data = Post.objects.all()
    return render(req, 'blog/list.html', {'data':data, 'username':username})
```

사용자가 ID에 입력했던 값을 session내 'username'키의 값으로 저장하였다.

또한 Blist에서는 session 중 'username'키에 해당 하는 정보를 가져와 username이라는 이름으로 저장하였으며, 이를 list.html에 username이라는 이름으로 넘겨주었다.

한편 Blist에서 세션 정보를 가져올 때 'username'키에 해당하는 값이 없는 경우를 대비해 `get()` 이용해 공백으로 default값을 설정하였다.

```html
<!-- /templates/blog/list.html 파일 -->
{% extends 'blog/base.html' %}

{% block content %}

    <h2> Posts </h2> <br>

    {{username}}님 환영합니다.<br>

    {% for d in data %}
        <a href='/blog/{{d.pk}}/detailPost'>{{d.title}}</a><br>
    {% endfor %}

{% endblock %}
```

세션이 유지되는 동안에는 로그인한 사용자의 이름이 나타나며, 세션정보가 없는(비로그인, 세션 종료 등) 상태로 .../blog/Blist/ 접속 시 guest로 표현된다.

* 포스트 추가 기능

로그인 한 사용자가 글을 작성할 수 있는 기능을 추가한다. 포스트를 DB에 저장하기 위해서는 title, text, auther 정보가 필요하다.

```html
<!-- /templates/blog/add.html 파일 -->
<form action='add' method='POST'>

    {% csrf_token %}
    제목: <input type=text name=title><br>
    본문: <textarea rows=10 cols=30 name=text></textarea><br>
    <input type=submit value='작성'>
    
</form>
```

`<form action='add' method='POST'>`: 포스트를 작성한 후 submit 버튼을 누르면 다시 add 페이지로 이동해 POST 방식으로 작성 내용으로 전송한다라는 의미이다.

POST 방식으로 전송하기 위해 `{% csrf_token %}` 

```python
# /blog/views.py 파일
from django.contrib.auth.models import User

class PostView(View):
    def get(self, request):
        return render(request, 'blog/add.html')
    
    def post(self, request):
        title = request.POST.get('title', '제목없음')
        text = request.POST.get('text')
        username = request.session['username']
        user = User.objects.get(username=username)
        Post.objects.create(title=title, text=text, author=user)
        return redirect('/blog/Blist')
```

GET 방식으로 요청이 들어왔을 때는 add.html 렌더링한다. 이는 글을 작성하기 위해 add 페이지에 접속했을 때이다.

글을 작성한 후 submit 버튼을 누르면 POST 방식으로 add 페이지에 접속하게 된다. 그러면 sever에서는 title, text, username 값을 확인한다.

`user = User.objects.get(username=username)`: username 값과 일치하는 User DB내에 데이터를 호출.

`Post.objects.create(title=title, text=text, author=user)`: post 테이블(blog_post)에 이전까지 얻을 정보를 가지고 create 한다.

```python
# /blog/urls.py 파일
urlpatterns = [
    path('', views.index),
    # path('<name>', views.index2),
    path('<int:pk>/detail', views.index3),
    path('Blist/', views.Blist),
    path('<int:pk>/detailPost/', views.detailPost),
    path('klass/', views.klass.as_view()),
    path('login/', views.LoginView.as_view()),
    path('add/', views.PostView.as_view()),
]
```

* 포스트 수정 기능

기존에 등록된 포스트를 수정하는 기능을 추가. 이를 위해서는 먼저 기존의 포스트를 보여준 뒤 이를 수정할 수 있어야 한다.

html 파일을 디테일하게 코딩했던것과 달리 이번에는 Djang 내부 라이브러리를 이용하여 form을 만든다.

```python
# /blog/views.py 파일
from django.forms import Form, CharField, Textarea

class PostForm(Form):
    title = CharField(label='제목', max_length=20)
    text = CharField(label='본문', widget=Textarea)
    
class PostEditView(View):
    def get(self, request, pk):
        post = get_object_or_404(Post, pk=pk)
        form = PostForm(initial={'title':post.title, 'text':post.text})
        return render(request, 'blog/edit.html', {'form':form})
```

`from django.forms ...  widget=Textarea)`: django의 form 라이브러리를 사용하였다. 또한 이를 PostForm이라는 class로 객체화 하였다.

`form = PostForm(initial={'title':post.title, 'text':post.text})`: PostForm의 초기 값은 DB에 저장된(이전에 사용자가 작성한) 내용 그대로 보여준다.

```python
# /blog/urls.py 파일
urlpatterns = [
    path('', views.index),
    # path('<name>', views.index2),
    path('<int:pk>/detail', views.index3),
    path('Blist/', views.Blist),
    path('<int:pk>/detailPost/', views.detailPost),
    path('klass/', views.klass.as_view()),
    path('login/', views.LoginView.as_view()),
    path('add/', views.PostView.as_view()),
    path('<int:pk>/edit/', views.PostEditView.as_view()),
]
```

```html
<form action='.' method='POST'>
    {% csrf_token %}
    {{form.as_p}}
    <input type=submit value='수정'>
</form>
```

`action='.'`: 현재 주소로 다시 보낸다. 즉 GET 방식으로 접근한 .../1/edit에서 submit 버튼을 누르면 POST 방식으로 .../1/edit에 접근한다.

`{{form.as_p}}`: form.as_p는 각각의 폼 필드와 라벨을 함께 문단(역자 주: paragraph, 즉 `<p>` 태그)으로 감싸서 출력합니다. 예제 템플릿의 출력은 다음과 같습니다. [인용: Django 문서](https://django-doc-test-kor.readthedocs.io/en/old_master/topics/forms/index.html)

* 로그인한 사용자가 작성한 글만 보이도록 변경

```python
# /blog/views.py 파일
def Blist(req):
    username = req.session.get('username', '')
    user = User.objects.get(username = username)
    data = Post.objects.all().filter(author=user)
    return render(req, 'blog/list.html', {'data':data, 'username':username})
```

`Post.objects.all().filter(author=user)`: DB의 Post 테이블에서 author가 username 데이터만 호출.

* 수정 시 기준을 만족하지 못하면(ex. 제목 미기입 등) 에러 메세지와 함께 저장하지 못하도록 만들기.

```python
from django.core.exceptions import ValidationError

def TitleValidator(value):
    if len(value) < 5:
        raise ValidationError('제목이 너무 짧습니다.')

class PostForm(Form):
    title = CharField(label='제목', max_length=20, validators=[TitleValidator]) # 여기
    text = CharField(label='본문', widget=Textarea)
    
class PostEditView(View):
    def get(self, request, pk):
        post = get_object_or_404(Post, pk=pk)
        form = PostForm(initial={'title':post.title, 'text':post.text})
        return render(request, 'blog/edit.html', {'form':form})
    
    def post(self, request, pk):
        form = PostForm(request.POST) # 여기
        if form.is_valid(): # 여기
            post = get_object_or_404(Post, pk=pk)
            post.title, post.text = form['title'].value(), form['text'].value()
            post.publish()
            return redirect('/blog/Blist')
        return render(request, 'blog/edit.html', {'form':form, 'pk':pk}) # 여기
```

매우 피곤하니 자세한 설명은 생략한다!!!